### Below is the final data cleaning model.
- To use this model on your laptop, you have to modified paths to each CSV file.
- Code for each working component is in the *Components* folder, you can use it if you don't get what this model does, as
  well as using it for understanding, debugging and extending.
- The first cell of code is to extract and convert post data into a dataframe:
    * Split the single column with all the information into multiple columns with individual information.
    * Group and re-order entire dataframe according to topic tags.
    * Combine multiple elements of actual topic tags into one and fill the gaps where empty elements appears.
    * Drop unwanted columns, remove garbled characters, and replace characters with digits.
- The second cell of code is to extract and convert post impression data, details are almost the same as above, except calculating the ratio.
- The third cell of code is to extract and convert reaction summary data.
- The fourth cell of code is to extract and convert customer data and search data, and merge them to a single dataframe.
- The final cell of code is to merge all the dataframes above into a single dataframe, drop unwanted columns, merge some information and generate the csv files of the final data.

In [1]:
import string
import numpy
import pandas as pd

from UNiDAYS.Final_Model.myFunctions import *
# from UNiDAYS.Yixuan.Data_Processing.myFunctions import findAndReplace
# from UNiDAYS.Yixuan.Data_Processing.myFunctions import combineMultipleElem
# from UNiDAYS.Yixuan.Data_Processing.myFunctions import shiftElementPosMult
# from UNiDAYS.Yixuan.Data_Processing.myFunctions import shiftElementPosSingle
# from UNiDAYS.Yixuan.Data_Processing.myFunctions import cleanData
# from UNiDAYS.Yixuan.Data_Processing.myFunctions import calculateRatio

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)

ModuleNotFoundError: No module named 'UNiDAYS'

In [167]:
# V1
# Post_data_csv = r'/Volumes/UNiDAYS/postdata.csv'
#
# # Post impression data needs to be loaded into dataframe and store as a csv file and reload again
# # To avoid weird bugs
# Post_impression_data_json = r'/Volumes/UNiDAYS/EngagementData/PostImpressionsData.json'
# # You need to specify a path to store the csv file
# Post_impression_data_csv = r'/Users/xuyixuan/Desktop/temp1.csv'
#
# # Same as above
# Reaction_summery_data_json = r'/Volumes/UNiDAYS/EngagementData/ReactionsSummaryData.json'
# Reaction_summery_data_csv = r'/Users/xuyixuan/Desktop/temp2.csv'
#
# Customer_data_csv = r'/Volumes/UNiDAYS/customerdata.csv'
# Search_data_csv = r'/Volumes/UNiDAYS/searchdata.csv'
# Body_friName_data_csv = r'/Volumes/UNiDAYS/ProcessedData/body_and_friendly_name_analysis.csv'
#
# FINAL_DATA_DESTINATION = r'/Users/xuyixuan/Desktop/temp3.csv'



# V2
Post_data_csv = r'/Volumes/UNiDAYS/V2/postdata.csv'

# Post impression data needs to be loaded into dataframe and store as a csv file and reload again
# To avoid weird bugs
Post_impression_data_json = r'/Volumes/UNiDAYS/V2/EngagementData/PostImpressionsData.json'
# You need to specify a path to store the csv file
Post_impression_data_csv = r'/Users/xuyixuan/Desktop/temp4.csv'

# Same as above
Reaction_summery_data_json = r'/Volumes/UNiDAYS/V2/EngagementData/ReactionsSummaryData.json'
Reaction_summery_data_csv = r'/Users/xuyixuan/Desktop/temp5.csv'

Customer_data_csv = r'/Volumes/UNiDAYS/V2/customerdata.csv'
Search_data_csv = r'/Volumes/UNiDAYS/V2/searchdata.csv'
Body_friName_data_csv = r'/Volumes/UNiDAYS/ProcessedData/body_and_friendly_name_analysis.csv'

FINAL_DATA_DESTINATION = r'/Users/xuyixuan/Desktop/temp6.csv'


In [168]:
data = pd.read_csv(Post_data_csv, error_bad_lines=False)
# data = data["data"].str.split(', "', n = -1, expand = True)
data = data[data.columns[1]].str.split(', "', n = -1, expand = True)
data = data.loc[data[2] == 'PostType": 1']
# only keep postType = 1

# separate data with actual topicTags into one data frame
data_actual_topicTags = data.loc[(data[5] != 'TopicTags": null') & (data[5] != 'TopicTags": []')]
# separate data with null or empty topicTags into another data frame
data_empty_topicTags = data.loc[(data[5] == 'TopicTags": null') | (data[5] == 'TopicTags": []')]

# combine all topic tags into one element
combineMultipleElem(data_actual_topicTags, 5, ']', 5)
# switch columns from back to the front
shiftElementPosMult(data_actual_topicTags, 6, 'VersionId')
# concatenate dataframe with actual topic tags with dataframe with empty topic tags to reform the original dataframe
post_data = pd.concat([data_actual_topicTags, data_empty_topicTags])

# fill the gap
for row in range(len(post_data)):
    shiftElementPosSingle(post_data, row, 7, "CreativeId")
    shiftElementPosSingle(post_data, row, 16, 'Title": ')

post_data.drop(post_data.iloc[:, 25:33], inplace = True, axis = 1)
post_data.drop(post_data.columns[[2, 3, 4, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]], axis = 1, inplace= True)
post_data = post_data.reset_index(drop=True)
# reset row index
post_data.columns = range(post_data.shape[1])
# reset column index

# transform country code into numbers
for row in range(len(post_data)):
    findAndReplace(post_data, post_data, "US", "1", row, 4, 4)  # United State
    findAndReplace(post_data, post_data, "CA", "2", row, 4, 4)  # Canada
    findAndReplace(post_data, post_data, "IE", "3", row, 4, 4)  # Ireland
    findAndReplace(post_data, post_data, "GB", "4", row, 4, 4)  # Great Britain
    findAndReplace(post_data, post_data, "AU", "5", row, 4, 4)  # Australia
    findAndReplace(post_data, post_data, "NZ", "6", row, 4, 4)  # New Zealand
    findAndReplace(post_data, post_data, "NL", "7", row, 4, 4)  # Netherlands
    findAndReplace(post_data, post_data, "IT", "8", row, 4, 4)  # Italy
    findAndReplace(post_data, post_data, "DK", "9", row, 4, 4)  # Denmark
    findAndReplace(post_data, post_data, "AT", "10", row, 4, 4) # Austria
    findAndReplace(post_data, post_data, "DE", "11", row, 4, 4) # Germany
    findAndReplace(post_data, post_data, "BE", "12", row, 4, 4) # Belgium
    findAndReplace(post_data, post_data, "CH", "13", row, 4, 4) # Swiss
    findAndReplace(post_data, post_data, "L", "14", row, 4, 4)  # Luxembourg
    findAndReplace(post_data, post_data, "FR", "15", row, 4, 4) # France
    findAndReplace(post_data, post_data, "FI", "16", row, 4, 4) # Finland
    findAndReplace(post_data, post_data, "SG", "17", row, 4, 4) # Singapore
    findAndReplace(post_data, post_data, "ES", "18", row, 4, 4) # Spain
    findAndReplace(post_data, post_data, "E", "18", row, 4, 4)  # Spain
    findAndReplace(post_data, post_data, "O", "19", row, 4, 4)  # Unknown
    findAndReplace(post_data, post_data, "G", "20", row, 4, 4)  # Unknown

# remove garbled characters
post_data[0] = post_data[0].str.lstrip('{"')
post_data[2] = post_data[2].str.replace('"', '')

cleanData(post_data, string.ascii_letters, ' "[]:SN')
post_data.columns = ['Post Id', 'CanReact', 'Topic Tags', 'Customer Id', 'Cultural Code', 'Is Published', 'Publish Data', 'Last Modified On', 'Customer Region Id']

b'Skipping line 144: expected 9 fields, saw 22\nSkipping line 146: expected 9 fields, saw 22\n'


In [169]:
# read json file into dataframe with column contains actual information, and store it in a csv file
data = pd.read_json(Post_impression_data_json)
data = data["Items"]
data.to_csv(Post_impression_data_csv)

# read csv file into dataframe, if not do so will cause weired error
data = pd.read_csv(Post_impression_data_csv)
data = data["Items"].str.split(", '", n = -1, expand = True)

# remove unnecessary characters
data[1] = data[1].str.lstrip("engagement_breakdown': ")
data[1] = data[1].str.lstrip("{'M': {'")

impression_data = pd.DataFrame(columns = ['Post Id', 'Impressions', 'Engagements', 'Partner Header Clicked', 'Content Card Clicked', 'Shoppable Clicked', 'Content Card Poll Completed'])
impression_data['Impressions'] = data[0]

for row in range (len(data)):
    for column in range(1, len(data.columns)):
        findAndReplace(data, impression_data, 'post_id', data.iat[row, column], row, column, 0)
        findAndReplace(data, impression_data, 'engagements', data.iat[row, column], row, column, 2)
        findAndReplace(data, impression_data, 'partner_header_clicked', data.iat[row, column], row, column, 3)
        findAndReplace(data, impression_data, 'content_card_clicked', data.iat[row, column], row, column, 4)
        findAndReplace(data, impression_data, 'shoppable_clicked', data.iat[row, column], row, column, 5)
        findAndReplace(data, impression_data, 'content_card_poll_completed', data.iat[row, column], row, column, 6)
        
cleanData(impression_data, string.ascii_letters + "_{ '", " '{}:SN")

# initialise the Ratio column, calculate the result and put it in the Ratio column
impression_data['Ratio'] = 'default value'
for row in range(len(impression_data)):
    impression_data.iat[row, 7] = calculateRatio(int(impression_data.iat[row, 1]), int(impression_data.iat[row, 2]))

impression_data.drop(impression_data.iloc[:, 3:7], inplace = True, axis = 1)

In [170]:
data = pd.read_json(Reaction_summery_data_json)
data = data['Items']
data.to_csv(Reaction_summery_data_csv)
data = pd.read_csv(Reaction_summery_data_csv)
data = data['Items'].str.split(", '", n = -1, expand = True)

data[0] = data[0].str.lstrip("{'reactions': ")
data[0] = data[0].str.lstrip("{'M': {'")

reaction_data = pd.DataFrame(columns = ['Post Id', 'Yeah!', 'Lit', 'Meh', 'Huh', 'Yikes'])
reaction_data['Post Id'] = data[0]

for row in range (len(data)):
    for column in range(len(data.columns)):
        findAndReplace(data, reaction_data, "post_id", data.iat[row, column],row, column, 0)
        findAndReplace(data, reaction_data, "r5", data.iat[row, column], row, column, 1)
        findAndReplace(data, reaction_data, "r4", data.iat[row, column], row, column, 2)
        findAndReplace(data, reaction_data, "r3", data.iat[row, column], row, column, 3)
        findAndReplace(data, reaction_data, "r2", data.iat[row, column], row, column, 4)
        findAndReplace(data, reaction_data, "r1", data.iat[row, column], row, column, 5)
        
cleanData(reaction_data, string.ascii_letters + "_{ '012345", " '{}:SN")

In [171]:
customer_data = pd.read_csv(Customer_data_csv, header=None, names=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k'])
search_data = pd.read_csv(Search_data_csv, header=None, names=['Customer', 'Search Volumn'])
search_data.to_csv(r'/Users/xuyixuan/Desktop/check0.csv')

combineMultipleElem(customer_data, 2, ']', 2)

data = pd.DataFrame(columns=['Customer Region Id', 'Customer', 'Category'])
data['Customer Region Id'] = customer_data['a']
data['Customer'] = customer_data['b']
data['Category'] = customer_data['c']

data['Customer'] = data['Customer'].str.lstrip(string.ascii_letters + ' ')
search_data['Customer'] = search_data['Customer'].str.lstrip(string.ascii_letters + ' ')

combine_search_and_customer_data = pd.merge(data, search_data, on='Customer')
combine_search_and_customer_data.to_csv(r'/Users/xuyixuan/Desktop/check.csv')

In [172]:
#!/usr/bin/env python3

import csv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from emoji import UNICODE_EMOJI

Body_friName_data_csv = r'/Volumes/UNiDAYS/V2/result.csv'

def is_emoji(str):
    """
    check if a string contains emoji
    :param str: the tested string
    :return: the number of emoji the string have
    """
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str.count(emoji)
    return int(count)


with open(Post_data_csv, newline='', encoding='ISO8859') as csv_file:
    with open(Body_friName_data_csv, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Post Id", "FCompound", "FIfContain%", "FIfContainCustomer", "FIfContainDollors", "FIfContainEmoji",
                         "BodyCompound", "BodyIfContain%", "BodyIfContainCustomer", "BodyIfContainDollors",
                         "BodyIfContainEmoji"])

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            elif not len(row):
                line_count += 1
            else:
                str1 = "FriendlyName"
                str2 = "LastUpdatedBy"
                Fbegin = row[1].index(str1) + len('FriendlyName": "')
                Fend = row[1].index(str2) - len('", "')
                # friendly name
                str3 = row[1][Fbegin:Fend]
                sid = SentimentIntensityAnalyzer()
                ss = sid.polarity_scores(str3)
                judge1 = "%" in str3
                judge2 = "Customer" in str3
                judgeFDollors = "$" in str3
                judgeFEmoji = is_emoji(str3) >= 1

                str4 = '"Body"'
                str5 = "Link"
                str6 = "$type"
                if str4 in row[1]:
                    Bodybegin = row[1].index(str4) + len('"Body": "')

                    if row[1].index(str6) < row[1].index(str5):
                        Bodyend = row[1].index(str6) - len('", "')
                    else:
                        Bodyend = row[1].index(str5) - len('", "')

                    #body
                    strBody = row[1][Bodybegin:Bodyend]
                    if strBody == '':
                        writer.writerow(
                            [row[0], ss['compound'], judge1, judge2, judgeFDollors, judgeFEmoji, "NULL", "NULL", "NULL",
                             "NULL", "NULL"])
                    else:
                        judge3 = "%" in strBody
                        judge4 = "Customer" in strBody
                        judgeBodyEmoji = is_emoji(strBody) >= 1
                        judgeBodyDolllors = "$" in strBody
                        ss1 = sid.polarity_scores(strBody)
                        writer.writerow(
                            [row[0], ss['compound'], judge1, judge2, judgeFDollors, judgeFEmoji, ss1['compound'],
                             judge3, judge4, judgeBodyDolllors, judgeBodyEmoji])
                else:
                    writer.writerow(
                        [row[0], ss['compound'], judge1, judge2, judgeFDollors, judgeFEmoji, "NULL", "NULL", "NULL",
                         "NULL", "NULL"])


print ("write over")      

body_friName_data = pd.read_csv(Body_friName_data_csv)

store the result into body_friName_data;

In [173]:
final_data = pd.merge(post_data, combine_search_and_customer_data, on='Customer Region Id')
final_data = pd.merge(final_data, impression_data, on='Post Id')
final_data = pd.merge(final_data, reaction_data, on='Post Id')
final_data = pd.merge(final_data, body_friName_data, on='Post Id')

# since friendly name and body are both text, and they have basically the same features
# therefore we can combine same feature of friendly name and body together
# combination rule is, if any of the feature is true, then combined one is true, else combined one is false
for row in range(len(final_data)):
    if final_data.loc[row, 'FIfContain%'] != final_data.loc[row, 'BodyIfContain%']:
        final_data.loc[row, 'FIfContain%'] = True
    if final_data.loc[row, 'FIfContainCustomer'] != final_data.loc[row, 'BodyIfContainCustomer']:
        final_data.loc[row, 'FIfContainCustomer'] = True
    if final_data.loc[row, 'FIfContainDollors'] != final_data.loc[row, 'BodyIfContainDollors']:
        final_data.loc[row, 'FIfContainDollors'] = True
    if final_data.loc[row, 'FIfContainEmoji'] != final_data.loc[row, 'BodyIfContainEmoji']:
        final_data.loc[row, 'FIfContainEmoji'] = True

final_data = final_data.drop(['CanReact', 'Customer Id', 'Is Published', 'Publish Data', 'Last Modified On',
                    'Customer Region Id', 'Customer', 'FCompound', 'BodyIfContain%', 'BodyIfContainCustomer', 'BodyIfContainDollors',
                    'BodyIfContainEmoji', 'Topic Tags', 'Category'], axis=1)

final_data = final_data.rename(columns={'FIfContain%': 'IfContain%', 'FIfContainCustomer': 'IfContainCustomer',
                              'FIfContainDollors': 'IfContainDollers', 'FIfContainEmoji': 'IfContainEmoji'})

final_data[["Yeah!", "Lit", "Meh", "Huh", "Yikes"]].replace("", numpy.NaN)
final_data.fillna(0, inplace=True)

final_data.to_csv(FINAL_DATA_DESTINATION)
